In [10]:
import math
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def get_df(results_file: str):
	return pd.read_csv(results_file, index_col="workers")

In [44]:
def latency_vs_workers_box(df: pd.DataFrame):
	bp = plt.boxplot(df.transpose(), positions=df.index.tolist(), bootstrap=1000, showmeans=True, meanline=True, showfliers=False)
	plt.legend([bp['medians'][0], bp['means'][0]], ['median', 'mean'])
	plt.title("Latency vs Workers")
	plt.xlabel("Workers (processes)")
	plt.ylabel("Latency (ms)")
	plt.xticks(df.index.tolist(), labels=df.index.tolist())
	plt.grid()

def latency_vs_workers_plot(dfs: pd.DataFrame, logscale: bool):
	for (label, df), color in zip(dfs.items(), ("red", "green", "blue")):
		plt.errorbar(df.index, df.mean(1), yerr=df.std(1), color=color, label=label, ecolor="black", capsize=4.0)
	plt.legend()
	plt.title("Latency vs Workers")
	plt.xlabel("Workers (processes)")
	plt.ylabel("Latency (ms)")
	plt.xticks(df.index.tolist(), labels=df.index.tolist())
	if logscale:
		plt.yscale("log", subs=list(range(10)))
	plt.grid()

In [40]:
names = ["pump"]  # ["demo", "pump", "temperature", "ultra"]
sizes = [100000, 1000000, 10000000]  # [10, 100, 1000, 1000000, 1000000]
trials = 50
for name in names:
	cfg_file = Path(f"../cfgs/{name}.pickle")
	for size in sizes:
		out = Path(f"output/{name}/{size}/{trials}.csv")
		df = get_df(out)
		latency_vs_workers_box(df)
		plt.savefig(out.with_suffix(".png"), dpi=800)
		plt.close()

In [46]:
names = ["pump"]  # ["demo", "pump", "temperature", "ultra"]
sizes = [100000, 1000000, 10000000]  # [10, 100, 1000, 1000000, 1000000]
trials = 50
size_to_bytes = {100000: "800K", 1000000: "8.6M", 10000000: "86M"}
for name in names:
	cfg_file = Path(f"../cfgs/{name}.pickle")
	dfs = {}
	for size in sizes:
		out = Path(f"output/{name}/{size}/{trials}.csv")
		dfs[size_to_bytes[size]] = get_df(out)
	for logscale in (True, False):
		out = Path(f"output/{name}/{trials}_{logscale}.csv")
		latency_vs_workers_plot(dfs, logscale)
		plt.savefig(out.with_suffix(".png"), dpi=800)
		plt.close()